<a href="https://colab.research.google.com/github/KhangTheKangaroo/Text-Files-to-QnA-with-ChatBot/blob/main/Files_to_QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q langchain-openai==0.1.9
!pip install -q chainlit==1.1.304
!npm install -g localtunnel
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio

In [ ]:
!pip install chainlit --upgrade

In [ ]:
import torch
import chainlit as cl
from chainlit.types import AskFileResponse
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer , AutoModelForCausalLM , pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader , TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [ ]:
def process_file(file: AskFileResponse):
  if file.type == 'text/plain':
    Loader = TextLoader(file.path)
  elif file.type == 'application/pdf':
    Loader = PyPDFLoader

  loader = Loader(file.path)
  documents = loader.load()
  docs = text_splitter.split_documents(documents)
  for i, doc in enumerate(docs):
    doc.metadata['Source'] = f'Source_{i}'

  return docs

In [ ]:
def getvector_db(file: AskFileResponse):
  docs = process_file(file)
  cl.user_session.set ('docs', docs)
  vector_db = Chroma.from_documents(docs, embedding)

  return vector_db

In [ ]:
welcome_message = """Welcome to the Project! To get started:
1. Upload a PDF File or txt file
2. Ask a question about said file"""

In [ ]:
@cl.on_chat_start
async def on_chat_start():
  files = None
  while files is None:
    files = await cl.AskFileMessage (content = welcome_message, accept = ['text/plain', 'application/pdf'], max_size_mb = 20, timeout = 180).send()
  file = files [0]

  msg = cl.Message(content = f"Process '{file.name}'...", disable_feedback = True)
  await msg.send()

  vector_db = await cl.make_async(get_vector_db)(file)
  message_history = ChatMessageHistory()
  memory = ConversationBufferMemory (memory_key = 'chat_history', output_key = 'answer', chat_memory = message_history, return_messages = True)

  retriever = vector_db.as_retriver(search_type = 'mmr', search_kwargs = {'k': 3})

  chain = ConversationalRetrievalChain.from_llm(llm = LLM, chain_type = 'stuff', retriever = retriever, memory = memory, return_source_documents = True)
  msg.content = f"'{file.name}' processed. You can now ask question!"
  await msg.send()

  cl.user.session.set('chain', chain)

In [ ]:
@cl.on_message
async def on_message(message: cl.Message):
  chain = cl.user.session.get('chain')
  cb = cl.AsyncLangchainCallbackHandler()
  res = await chain.ainvoke(message.content, callback = [cb])
  answer = res['answer']
  source_documents = res['source_documents']
  text_elements = []

  if source_documents:
    for source_idx, source_doc in enumerate (source_documents):
      source_name = f'source{source_idx}'
      text_elements.append(cl.Text(content = source_doc.page_content, name = source_name))
      source_names = [text_el.name for text_el in text_elements]
      if source_names :

        answer += f"\ nSources : { ' , '.join(source_names)}"
      else:

        answer += "\nNo sources found"

    await cl.Message (content = answer , elements = text_elements ).send ()

In [ ]:
!chainlit run app.py --host 0.0.0.0 --port 8000 &>/content/logs.txt &

In [ ]:
import urllib

In [ ]:
rint ("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read ().decode('utf8').strip("\n"))
!lt --port 8000 -- subdomain aivn-simple-ra